In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
df_temp = pd.read_csv("../meteorology/dados_temperatura.csv")
df_prec = pd.read_csv("../meteorology/dados_precipitacao.csv")
df_energy = pd.read_csv("data/shared_1year.csv")

In [ ]:
temp_cols = df_temp.columns
prec_cols = df_prec.columns

## IPMA Temperature

In [ ]:
df_temp

## IPMA Precipitation

In [ ]:
df_prec

## Created Energy Dataset

In [ ]:
df_energy

## Preprocessing

In [ ]:
print("NaN values:\n",df_temp.isna().sum())
print("Negative values:",(df_temp.values[:,1:] < 0).sum())

In [ ]:
print("NaN values:\n",df_prec.isna().sum())
print("Negative values:",(df_prec.values[:,1:] < 0).sum())

In [ ]:
print("NaN values:\n",df_energy.isna().sum())
print("Negative values:",(df_energy.values[:,1:] < 0).sum())

### Equalize intervals

In [2]:
def resample_dataset_15_minutes(df):
    df['Time'] = pd.to_datetime(df['Time'])
    start = time.time()
    df_ind = df.set_index('Time')
    df_split = np.array_split(df_ind, len(df_ind)/6)
    df_time = df_ind.resample('15T').apply(lambda x: x[0])
    df_split15 = np.array_split(df_time, len(df_time)/4)
    for hour in range(len(df_split)):
        for col in range(df_time.shape[1]):
            df_split15[hour].iloc[0,col] = df_split[hour].iloc[0,col]
            df_split15[hour].iloc[1,col] = (df_split[hour].iloc[1,col] + df_split[hour].iloc[2,col]) / 2
            df_split15[hour].iloc[2,col] = df_split[hour].iloc[3,col]
            df_split15[hour].iloc[3,col] = (df_split[hour].iloc[4,col] + df_split[hour].iloc[5,col]) / 2
    df_concat = pd.DataFrame(np.concatenate(df_split15))
    dataframe = pd.DataFrame()
    dataframe.index = pd.date_range(start='2020-10-01', end='2023-04-01', closed='left', freq='15T')
    dataframe['Time'] = dataframe.index
    dataframe = dataframe.reset_index(drop=True)
    df_resampled = pd.concat([dataframe, df_concat], axis=1)
    print('Elapsed time: %.4f seconds' % (time.time() - start))
    return df_resampled

In [ ]:
df_resampled = resample_dataset_15_minutes(df_temp)
df_resampled.columns = temp_cols
df_resampled

In [ ]:
df_resampled.to_pickle("data/df_temperature.pkl")

In [ ]:
df_resampled_prec = resample_dataset_15_minutes(df_prec)
df_resampled_prec.columns = prec_cols
df_resampled_prec

In [ ]:
df_resampled_prec.to_pickle("data/df_precipitation.pkl")

## Data with more metrics

In [ ]:
station_viseu = pd.read_csv('../meteorology/dados_01200560.csv')
station_aveiro = pd.read_csv('../meteorology/dados_01210702.csv')
station_viseu_cols = station_viseu.columns
station_aveiro_cols = station_aveiro.columns

In [ ]:
station_viseu

In [ ]:
station_aveiro

In [ ]:
print("Station Viseu")
print("NaN values:",station_viseu.isna().sum().sum())
print("Negative values:\n",(station_viseu.iloc[:,2:] < 0).sum())
print("\nStation Aveiro")
print("NaN values:",station_aveiro.isna().sum().sum())
print("Negative values:\n",(station_aveiro.iloc[:,2:] < 0).sum())

In [ ]:
station_viseu_resample = resample_dataset_15_minutes(station_viseu)
station_viseu_resample.drop(0, inplace=True, axis=1)
station_viseu_resample.columns = station_viseu_cols[1:]
station_viseu_resample

In [ ]:
station_aveiro_resample = resample_dataset_15_minutes(station_aveiro)
station_aveiro_resample.drop(0, inplace=True, axis=1)
station_aveiro_resample.columns = station_aveiro_cols[1:]
station_aveiro_resample

In [ ]:
st_viseu_intersect = station_viseu_resample.iloc[38017:73057].reset_index(drop=True)
print("Nan values:", st_viseu_intersect.isna().sum().sum())
print("Negative values:",(st_viseu_intersect.values[:,1:] < 0).sum())
st_viseu_intersect

In [ ]:
station_viseu_resample.to_pickle("data/station_viseu_all.pkl")
st_viseu_intersect.to_pickle("data/station_viseu.pkl")

In [ ]:
st_av_intersect = station_aveiro_resample.iloc[38017:73057].reset_index(drop=True)
print("Nan values:", st_av_intersect.isna().sum().sum())
print("Negative values:",(st_av_intersect.values[:,1:] < 0).sum())
st_av_intersect

In [ ]:
station_aveiro_resample.to_pickle("data/station_aveiro_all.pkl")
st_av_intersect.to_pickle("data/station_aveiro.pkl")

## Merge with Shared Energy Dataset

In [3]:
station_aveiro_resample = pd.read_pickle("data/station_aveiro_all.pkl")
station_viseu_resample = pd.read_pickle("data/station_viseu_all.pkl")
station_aveiro_resample

,Time,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total
0,2020-10-01 00:00:00,16.80,16.90,16.6,167.0,179.0,1.40,2.40,0.0,0.0
1,2020-10-01 00:15:00,16.65,16.85,16.5,172.0,165.5,1.55,2.45,0.0,0.0
2,2020-10-01 00:30:00,16.50,16.60,16.4,171.0,175.0,1.20,2.30,0.0,0.0
3,2020-10-01 00:45:00,16.40,16.60,16.3,167.0,171.5,1.20,2.30,0.0,0.0
4,2020-10-01 01:00:00,16.40,16.50,16.4,163.0,172.0,1.30,2.50,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
87547,2023-03-31 22:45:00,-990.00,-990.00,-990.0,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87548,2023-03-31 23:00:00,-990.00,-990.00,-990.0,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87549,2023-03-31 23:15:00,-990.00,-990.00,-990.0,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87550,2023-03-31 23:30:00,-990.00,-990.00,-990.0,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0


In [5]:
df_energy = pd.read_csv("data/loureiro_40casas.csv")
df_energy

,Time,Energy_1,Energy_2,Energy_9,Energy_15,Energy_16,Energy_17,Energy_30,Energy_35,Energy_48,...,Energy_138,Energy_139,Energy_142,Energy_146,Energy_148,Energy_151,Energy_159,Energy_160,Energy_161,Energy_164
0,2022-05-05 12:00:00,0.068,0.018,0.024,0.164,0.072,0.055,0.006,0.080,0.062,...,0.052,0.083,0.084,0.068,0.060,0.042,0.055,0.069,0.010,0.004
1,2022-05-05 12:15:00,0.070,0.018,0.024,0.135,0.058,0.060,0.006,0.056,0.062,...,0.052,0.053,0.004,0.048,0.078,0.053,0.055,0.063,0.010,0.003
2,2022-05-05 12:30:00,0.122,0.018,0.024,0.121,0.011,0.096,0.006,0.052,0.062,...,0.052,0.035,0.004,0.062,0.061,0.057,0.055,0.049,0.010,0.004
3,2022-05-05 12:45:00,0.132,0.018,0.024,0.138,0.010,0.064,0.006,0.052,0.062,...,0.052,0.082,0.004,0.056,0.125,0.034,0.055,0.097,0.010,0.004
4,2022-05-05 13:00:00,0.125,0.018,0.024,0.104,0.011,0.072,0.006,0.053,0.062,...,0.052,0.076,0.004,0.034,0.070,0.038,0.055,0.075,0.010,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38014,2023-06-07 01:15:00,0.120,0.020,0.066,0.120,0.003,0.094,0.037,0.045,0.071,...,0.047,0.054,0.004,0.080,0.014,0.028,0.066,0.058,0.000,0.003
38015,2023-06-07 01:30:00,0.121,0.004,0.057,0.087,0.002,0.077,0.036,0.046,0.059,...,0.051,0.038,0.004,0.060,0.028,0.023,0.067,0.032,0.016,0.004
38016,2023-06-07 01:45:00,0.158,0.007,0.056,0.086,0.003,0.087,0.021,0.045,0.059,...,0.068,0.045,0.004,0.057,0.029,0.043,0.065,0.495,0.008,0.003
38017,2023-06-07 02:00:00,0.102,0.042,0.056,0.085,0.002,0.072,0.027,0.045,0.059,...,0.067,0.053,0.003,0.061,0.012,0.047,0.041,0.274,0.000,0.004


In [6]:
dateindex = df_energy['Time']
df = df_energy.drop(['Time'], axis=1)
st_av = station_aveiro_resample.drop(['Time'], axis=1)
print(df.shape)
house_list = []
test = dateindex
for i in df:
    test = pd.concat([dateindex, df[i]], axis=1)
    test = test.rename(columns={i: 'Energy'})
    test['Location'] = i
    test = pd.concat([test, st_av], axis=1)
    house_list.append(test)
df_joined = house_list[0]
for i in house_list[1:]:
    df_joined = pd.concat([df_joined, i], axis=0)
df_joined.dropna(inplace=True)
df_joined.reset_index(drop=True, inplace=True)
df_joined

(38019, 40)


,Time,Energy,Location,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total
0,2022-05-05 12:00:00,0.068,Energy_1,16.80,16.90,16.60,167.0,179.0,1.40,2.40,0.00,0.0
1,2022-05-05 12:15:00,0.070,Energy_1,16.65,16.85,16.50,172.0,165.5,1.55,2.45,0.00,0.0
2,2022-05-05 12:30:00,0.122,Energy_1,16.50,16.60,16.40,171.0,175.0,1.20,2.30,0.00,0.0
3,2022-05-05 12:45:00,0.132,Energy_1,16.40,16.60,16.30,167.0,171.5,1.20,2.30,0.00,0.0
4,2022-05-05 13:00:00,0.125,Energy_1,16.40,16.50,16.40,163.0,172.0,1.30,2.50,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1520755,2023-06-07 01:15:00,0.003,Energy_164,16.40,16.50,16.30,340.0,338.0,2.50,3.40,0.00,0.0
1520756,2023-06-07 01:30:00,0.004,Energy_164,16.40,16.50,16.35,335.0,338.5,2.05,2.90,0.05,0.0
1520757,2023-06-07 01:45:00,0.003,Energy_164,16.50,16.50,16.40,333.0,313.0,1.80,2.70,0.10,0.3
1520758,2023-06-07 02:00:00,0.004,Energy_164,16.50,16.60,16.40,337.5,342.0,2.20,2.90,0.05,0.0


In [7]:
energy = df_joined['Energy']
df_joined.drop(['Energy'], axis=1, inplace=True)
df_joined = pd.concat([df_joined, energy], axis=1)
df_joined

,Time,Location,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total,Energy
0,2022-05-05 12:00:00,Energy_1,16.80,16.90,16.60,167.0,179.0,1.40,2.40,0.00,0.0,0.068
1,2022-05-05 12:15:00,Energy_1,16.65,16.85,16.50,172.0,165.5,1.55,2.45,0.00,0.0,0.070
2,2022-05-05 12:30:00,Energy_1,16.50,16.60,16.40,171.0,175.0,1.20,2.30,0.00,0.0,0.122
3,2022-05-05 12:45:00,Energy_1,16.40,16.60,16.30,167.0,171.5,1.20,2.30,0.00,0.0,0.132
4,2022-05-05 13:00:00,Energy_1,16.40,16.50,16.40,163.0,172.0,1.30,2.50,0.00,0.0,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...
1520755,2023-06-07 01:15:00,Energy_164,16.40,16.50,16.30,340.0,338.0,2.50,3.40,0.00,0.0,0.003
1520756,2023-06-07 01:30:00,Energy_164,16.40,16.50,16.35,335.0,338.5,2.05,2.90,0.05,0.0,0.004
1520757,2023-06-07 01:45:00,Energy_164,16.50,16.50,16.40,333.0,313.0,1.80,2.70,0.10,0.3,0.003
1520758,2023-06-07 02:00:00,Energy_164,16.50,16.60,16.40,337.5,342.0,2.20,2.90,0.05,0.0,0.004


In [8]:
# Transform negative values to nan
tmp = df_joined.iloc[:,2:]
tmp[tmp < 0] = np.nan
df_joined.iloc[:,2:] = tmp
df_joined

,Time,Location,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total,Energy
0,2022-05-05 12:00:00,Energy_1,16.80,16.90,16.60,167.0,179.0,1.40,2.40,0.00,0.0,0.068
1,2022-05-05 12:15:00,Energy_1,16.65,16.85,16.50,172.0,165.5,1.55,2.45,0.00,0.0,0.070
2,2022-05-05 12:30:00,Energy_1,16.50,16.60,16.40,171.0,175.0,1.20,2.30,0.00,0.0,0.122
3,2022-05-05 12:45:00,Energy_1,16.40,16.60,16.30,167.0,171.5,1.20,2.30,0.00,0.0,0.132
4,2022-05-05 13:00:00,Energy_1,16.40,16.50,16.40,163.0,172.0,1.30,2.50,0.00,0.0,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...
1520755,2023-06-07 01:15:00,Energy_164,16.40,16.50,16.30,340.0,338.0,2.50,3.40,0.00,0.0,0.003
1520756,2023-06-07 01:30:00,Energy_164,16.40,16.50,16.35,335.0,338.5,2.05,2.90,0.05,0.0,0.004
1520757,2023-06-07 01:45:00,Energy_164,16.50,16.50,16.40,333.0,313.0,1.80,2.70,0.10,0.3,0.003
1520758,2023-06-07 02:00:00,Energy_164,16.50,16.60,16.40,337.5,342.0,2.20,2.90,0.05,0.0,0.004


In [9]:
print("NaN values:\n",df_joined.isna().sum())
print("Negative values:\n",(df_joined.iloc[:,2:] < 0).sum())

NaN values:
 Time                         0
Location                     0
Temp_Med                 11080
Temp_Max                 11080
Temp_Min                 11120
Rumo_Vento_Med           10160
Rumo_Vento_Max           10160
Intensidade_Vento_Med    10160
Intensidade_Vento_Max    10160
Precip                   23520
Rad_Total                11080
Energy                       0
dtype: int64
Negative values:
 Temp_Med                 0
Temp_Max                 0
Temp_Min                 0
Rumo_Vento_Med           0
Rumo_Vento_Max           0
Intensidade_Vento_Med    0
Intensidade_Vento_Max    0
Precip                   0
Rad_Total                0
Energy                   0
dtype: int64


In [10]:
# Number of records per location
df_joined.Location.value_counts()

Energy_1      38019
Energy_2      38019
Energy_118    38019
Energy_123    38019
Energy_125    38019
Energy_126    38019
Energy_127    38019
Energy_129    38019
Energy_132    38019
Energy_135    38019
Energy_138    38019
Energy_139    38019
Energy_142    38019
Energy_146    38019
Energy_148    38019
Energy_151    38019
Energy_159    38019
Energy_160    38019
Energy_161    38019
Energy_117    38019
Energy_110    38019
Energy_108    38019
Energy_58     38019
Energy_9      38019
Energy_15     38019
Energy_16     38019
Energy_17     38019
Energy_30     38019
Energy_35     38019
Energy_48     38019
Energy_60     38019
Energy_96     38019
Energy_65     38019
Energy_78     38019
Energy_86     38019
Energy_90     38019
Energy_91     38019
Energy_94     38019
Energy_95     38019
Energy_164    38019
Name: Location, dtype: int64

In [11]:
## Drop Radiation due to lack of samples in station viseu
#df_joined = df_joined.drop(['Rad_Total'], axis=1)
## Interpolate nan values
df_joined = df_joined.interpolate(method='pad', axis=0)
df_joined

,Time,Location,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total,Energy
0,2022-05-05 12:00:00,Energy_1,16.80,16.90,16.60,167.0,179.0,1.40,2.40,0.00,0.0,0.068
1,2022-05-05 12:15:00,Energy_1,16.65,16.85,16.50,172.0,165.5,1.55,2.45,0.00,0.0,0.070
2,2022-05-05 12:30:00,Energy_1,16.50,16.60,16.40,171.0,175.0,1.20,2.30,0.00,0.0,0.122
3,2022-05-05 12:45:00,Energy_1,16.40,16.60,16.30,167.0,171.5,1.20,2.30,0.00,0.0,0.132
4,2022-05-05 13:00:00,Energy_1,16.40,16.50,16.40,163.0,172.0,1.30,2.50,0.00,0.0,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...
1520755,2023-06-07 01:15:00,Energy_164,16.40,16.50,16.30,340.0,338.0,2.50,3.40,0.00,0.0,0.003
1520756,2023-06-07 01:30:00,Energy_164,16.40,16.50,16.35,335.0,338.5,2.05,2.90,0.05,0.0,0.004
1520757,2023-06-07 01:45:00,Energy_164,16.50,16.50,16.40,333.0,313.0,1.80,2.70,0.10,0.3,0.003
1520758,2023-06-07 02:00:00,Energy_164,16.50,16.60,16.40,337.5,342.0,2.20,2.90,0.05,0.0,0.004


In [12]:
df_joined.to_csv("data/loureiro_meteo.csv", index=False)

In [13]:
min(df_joined['Time']), max(df_joined['Time'])

('2022-05-05 12:00:00', '2023-06-07 02:15:00')